## (1) Load Processed Data in the Paper

Currently, we support norman / adamson / dixit.

In [1]:
!pip install -r /content/sample_data/requirements.txt

In [2]:
import sys
sys.path.append('../')

from gears import PertData

pert_data = PertData('./data') # specific saved folder
pert_data.load(data_name = 'norman') # specific dataset name
pert_data.prepare_split(split = 'simulation', seed = 1) # get data split with seed
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader

/usr/local/lib/python3.11/dist-packages/anndata/utils.py:434: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.11/dist-packages/anndata/utils.py:434: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.11/dist-packages/anndata/utils.py:434: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.11/dist-packages/anndata/utils.py:434: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.11/dist-packages/anndata/utils.py:434: FutureWarning: Importing read_mtx from `anndata` is deprecated. Import anndata.io.read_mtx instead.
  warnings.warn(msg, FutureWarning)
/usr

In [3]:
import sys
sys.path.append('../')

from gears import PertData

pert_data = PertData('./data') # specific saved folder
pert_data.load(data_name = 'dixit') # specific dataset name
pert_data.prepare_split(split = 'simulation', seed = 1) # get data split with seed
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader

Found local copy...
Downloading...
100%|██████████| 168M/168M [00:05<00:00, 31.8MiB/s]
Extracting zip file...
Done!
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['AURKC+ctrl']
Creating pyg object for each cell in the data...
Creating dataset file...
100%|██████████| 19/19 [01:14<00:00,  3.93s/it]
Done!
Saving new dataset pyg object at ./data/dixit/data_pyg/cell_graphs.pkl
Done!
Creating new splits....
Saving new splits at ./data/dixit/splits/dixit_simulation_1_0.75.pkl
Simulation split test composition:
combo_seen0:0
combo_seen1:0
combo_seen2:0
unseen_single:5
Done!
Creating dataloaders....
Done!


## (2) Create your own Perturb-Seq data
Prepare a scanpy adata object with
1. `adata.obs` dataframe has `condition` and `cell_type` columns, where `condition` is the perturbation name for each cell. Control cells have condition format of `ctrl`, single perturbation has condition format of `A+ctrl` or `ctrl+A`, combination perturbation has condition format of `A+B`.
2. `adata.var` dataframe has `gene_name` column, where each gene name is the gene symbol.
3. `adata.X` stores the post-perturbed gene expression.

Here an example using dixit 2016 dataset.

In [5]:
import scanpy as sc
# adata = sc.read_h5ad('sample_adata.h5ad')
adata = sc.read_h5ad('/content/data/dixit/perturb_processed.h5ad')

adata

AnnData object with n_obs × n_vars = 44735 × 5012
    obs: 'condition', 'cell_type', 'dose_val', 'control', 'condition_name'
    var: 'gene_name'
    uns: 'non_dropout_gene_idx', 'non_zeros_gene_idx', 'rank_genes_groups_cov_all', 'top_non_dropout_de_20', 'top_non_zero_de_20'
    layers: 'counts'

In [6]:
adata.var.head(2)

,gene_name
2,A1BG-AS1
7,AAK1


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('gene_name').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [7]:
adata.obs.head(2)

,condition,cell_type,dose_val,control,condition_name
TAACATGAAAAGTG_p7d_F3,CREB1+ctrl,K562,1+1,0,K562_CREB1+ctrl_1+1
CTATCCCTTGTGCA_p7d_G3,GABPA+ctrl,K562,1+1,0,K562_GABPA+ctrl_1+1


### Suggested normalization

For raw count data we recommend the following normalization and subsetting to the top 5000 most variable genes

In [8]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata,n_top_genes=5000, subset=True)

### Create dataloader

GEARS will take it from here. The new data processing takes around 15 minutes for 5K genes and 100K cells.

In [9]:
import sys
sys.path.append('../')

from gears import PertData

pert_data = PertData('./data') # specific saved folder
pert_data.new_data_process(dataset_name = 'dixit', adata = adata) # specific dataset name and adata object
pert_data.load(data_path = './data/dixit') # load the processed data, the path is saved folder + dataset_name
pert_data.prepare_split(split = 'simulation', seed = 1) # get data split with seed
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader

Found local copy...
Found local copy...
Creating pyg object for each cell in the data...
Creating dataset file...
 75%|███████▌  | 15/20 [01:22<00:26,  5.35s/it]

AURKC+ctrl


100%|██████████| 20/20 [01:31<00:00,  4.58s/it]
Done!
Saving new dataset pyg object at ./data/dixit/data_pyg/cell_graphs.pkl
Done!
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['AURKC+ctrl']
Local copy of pyg dataset is detected. Loading...
Done!
Local copy of split is detected. Loading...
Simulation split test composition:
combo_seen0:0
combo_seen1:0
combo_seen2:0
unseen_single:5
Done!
Creating dataloaders....
Done!


here1
